In [17]:
#imports
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [18]:
#training data
df_train=pd.read_csv('/content/drive/My Drive/minor/german_dataset/german_dataset.tsv',sep='\t', encoding="utf-8")
df_train=df_train[df_train.task_2!='NONE']

In [19]:
#training and validation split
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_train['text'], df_train['task_2'],test_size=0.15,random_state=140) 

In [20]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder() #label encoder
train_y = encoder.fit_transform(train_y) #encoding label for training data
valid_y = encoder.fit_transform(valid_y) #encoding label for validation data

In [21]:
#testing data
df_test=pd.read_csv('/content/drive/My Drive/minor/german_dataset/hasoc_de_test_gold.tsv',sep='\t', encoding="utf-8") 
df_test=df_test[df_test.task_2!='NONE']
test_y = encoder.fit_transform(df_test['task_2']) #encoding the label of testing data

In [22]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(df_train['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)  #get count vector features for training data
xvalid_count =  count_vect.transform(valid_x)  #get count vector features for validation data
x_count =  count_vect.transform(df_test['text'])  #get count vector features for testing data

In [23]:
def model(classifier, feature_vector_train, label, feature_vector_valid,feature_vector_test):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions_valid = classifier.predict(feature_vector_valid)

    # predict the labels on testing dataset
    predictions_test = classifier.predict(feature_vector_test)

    print("classification report for validation")
    print(metrics.classification_report(predictions_valid,valid_y))
    print("classification report for testing")
    print(metrics.classification_report(predictions_test,test_y))


In [24]:
# calling the xgboost classifier model for training ,validation and testing
model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc(),x_count.tocsc())

classification report for validation
              precision    recall  f1-score   support

           0       0.50      0.47      0.48        15
           1       0.78      0.58      0.67        43
           2       0.12      0.50      0.20         4

    accuracy                           0.55        62
   macro avg       0.47      0.52      0.45        62
weighted avg       0.67      0.55      0.59        62

classification report for testing
              precision    recall  f1-score   support

           0       0.10      0.44      0.16         9
           1       0.92      0.59      0.72       121
           2       0.06      0.17      0.08         6

    accuracy                           0.56       136
   macro avg       0.36      0.40      0.32       136
weighted avg       0.83      0.56      0.65       136

